In [239]:
import pandas as pd
import ast
from nltk.stem import SnowballStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 

In [ ]:
# PUT THE SNOWBALL STEMMER HERE

In [47]:
skill_set = ['r','python','java','c++','ruby','perl','matlab','javascript','scala','excel','tableau',
             'd3js','sas','spss','d3','hadoop','mapreduce','spark',
             'pig','hive','shark','zookeeper','flume','mahout',
             'sql','nosql','hase','cassandra','mongodb','docker','aws']

In [258]:
df = pd.read_csv('data_scientist.csv')

In [259]:
df.head()

,Unnamed: 0,city,company,desc,job_title,skills,state,url,job
0,0,Unite,IBM,"['job', 'description', 'ibm', 'amazing', 'oppo...",Entry-Level Data Scientist,"['r', 'java', 'sas', 'python']",States,https://www.indeed.com/rc/clk?jk=68907b23cb078...,Data Scientist
1,1,New York,Rent the Runway,"['rent', 'runway', 'recently', 'named', '9', '...",Data Scientist,"['java', 'r', 'python']",NY,https://www.indeed.com/rc/clk?jk=b59410cae51ba...,Data Scientist
2,2,Unite,IBM,"['internship', 'job', 'description', 'position...",Data Scientist - Intern Conversion,"['r', 'java', 'sas', 'python']",States,https://www.indeed.com/rc/clk?jk=a4139a3bfaa51...,Data Scientist
3,3,New York,JobRobin,"['100000', '170000', 'year', 'job', 'descripti...",Sports Data Scientist,"['r', 'sql', 'python']",NY,https://www.indeed.com/company/JobRobin/jobs/S...,Data Scientist
4,4,New York,Newforce,"['40', '50', 'hour', 'contract', 'job', 'summa...",Data Scientist,"['r', 'python']",NY,https://www.indeed.com/company/Newforce-Ltd-Pv...,Data Scientist


,Unnamed: 0,city,company,desc,job_title,skills,state,url,job


In [279]:
mle_df = pd.read_csv('machine_learning_engineer.csv')

In [286]:
mle_df.job_title.unique()

array(['Machine Learning Engineer - Center of Excellence - Associate',
       'Machine Learning Engineer',
       'Machine Learning Engineer – Free Mission',
       'Applied Machine Learning Engineer Intern',
       'Machine Learning Engineer, Cash',
       'Trulia: Machine Learning Engineer - Intern',
       'Data Scientist/Machine Learning Engineer',
       'Principle Machine Learning Engineer',
       'Machine Learning Engineer (CV focus)',
       'Algorithm & Machine Learning Eng',
       '2018 Internship – Machine Learning Engineer Intern',
       'Machine Learning Engineer - Core ML',
       'Machine Learning Engineer (NLP focus)',
       'Machine Learning Engineer - App Development Experience - Core ML',
       'Machine Learning Engineer - VP - Center of Excellence',
       'Staff Machine Learning Engineer (NLP)',
       'Machine Learning Engineer - Executive Director - Center of Excellence',
       'Siri - Machine Learning Engineer, Advanced Development',
       'NLP/Machine Le

In [151]:
def join_dfs(dfList):
    return reduce(lambda x, y: x.append(y), dfList)

In [87]:
def format_df(df):
    #make one big data frame and then get values for everything 
    
    #getting rid of first column check if you can just do index remove
    df = df.iloc[:,1:]
    
    #dropping duplicated rows
    df = df.drop_duplicates()
    
    #formatting for alternate locations 
    
    #some have on city or state because there are multiple available 
    df['city'] = df.city.apply(lambda x: 'Multiple' if x == "Unite" else x)
    df['city'] = df.city.apply(lambda x: 'unavailable' if x == "unavailabl" else x)
    
    df['state'] = df.state.apply(lambda x: 'Multiple' if x == "States" else x)
    df['state'] = df.state.apply(lambda x: 'MN' if x == 'Minnesota' else x)
    df['state'] = df.state.apply(lambda x: 'AZ' if x == 'Arizona' else x)
    
    #if location if unavaiable take into account when doing search DO THIS 
    # IF LOCATION IS REMOVE TAKE INTO ACCOUNT 
    
    #REPLACE NAN WITH UNAVAILABLE
  
    
    #getting rid of jobs that have no descriptions 
    index = df[df['desc'] == "['U', 'n', 'a', 'v', 'a', 'i', 'l', 'a', 'b', 'l', 'e']"].index
    df = mle_df.drop(index,axis = 0)
    
    return df 

In [39]:
#df.head()

In [50]:
def get_query_text(skill_set,query_url):
    
    _, _, _, job_description = get_job_info(query_url)
    text_no_skills, _ = clean_job_description(job_description,skill_set)
    
    return text_no_skills

In [256]:
def get_top_sim(df):
    
    query_words = [" ".join(query)]
    
    job_words = df.desc.values
    job_words = [ast.literal_eval(x) for x in job_words] #turns string into a list
    job_words[x for x in job_words if not isinstance(x, int)] #gets rid of numbers
    
    
    stemmer = SnowballStemmer('english') #get rid if this if you're going to do semantic similary
    job_words_stem = [] 
    for job in job_words:
        job_words_stem.append([stemmer.stem(word) for word in job]) 
    
    job_desc = [" ".join(li) for li in job_words_stem]
    
    vectorizer = TfidfVectorizer() #make sure these defaults are correct
    model = vectorizer.fit(job_desc)
    
    query_tfidf = model.transform(query_words)
    job_tfidf = model.transform(job_desc)
    
    cosine_sim = cosine_similarity(query_tfidf,job_tfidf)
    top_10_sim = cosine_sim.argsort()[0][::-1][0:10]
    
    return cosine_sim, top_10_sim
    
    

In [257]:
get_top_sim(df)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [42]:
def get_top_sim(query_words, df):
    #TO DO 1) here is where I need to incorporate semantic similiarty 
    #      2) can't stem prior to doing similiarty so need to unstem and then resteam
    #      3) HERE IS WHERE I NEED TO DO THE STEMMING
    

    
    query_words = [" ".join(query)]
    job_words = df.desc.apply(lambda x: " ".join(x)).values.tolist() #this gives you a list of list of the jobs 
     
    vectorizer = TfidfVectorizer() #make sure these defaults are correct
    model = vectorizer.fit(job_words)
    
    query_tfidf = model.transform(query_words)
    job_tfidf = model.transform(job_words)
    
    cosine_sim = cosine_similarity(query_tfidf,job_tfidf)
    top_10_sim = cosine_sim.argsort()[0][::-1][0:10]
    
    return cosine_sim, top_10_sim

In [43]:
def get_top_jobs(df,indices):
    return df.iloc[indices,:]

In [133]:
mle_df = mle_df.drop(index,axis = 0)

In [76]:
mle_df.iloc[407,:].url #make it so that if desc == unaviable then they do not exist 

'https://www.indeed.com/rc/clk?jk=1b16d093ee6951c5&fccid=1fea2a7d8e897821&vjs=3'